## **Import spark functions and libraries**

In [ ]:
from pyspark import *
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *



## **Data Reading**

In [ ]:
df = spark.read.format('parquet')\
             .option('inferschema', 'true')\
             .option('header','true')\
            .load('abfss://bronze@mynewadlsaccount.dfs.core.windows.net/carsales_rawdata')

display(df)

Branch_ID,Dealer_ID,Model_ID,Revenue,Units_sold,Date_Id,Day,Month,Year,Branch_Name,Dealer_Name


**Transformations**

In [ ]:
df= df.withColumn('Model_category', split(col('Model_ID'), '-')[0])\
             .withColumn('Rev_per_unit', col('Revenue')/col('Units_sold'))

display(df)

Branch_ID,Dealer_ID,Model_ID,Revenue,Units_sold,Date_Id,Day,Month,Year,Branch_Name,Dealer_Name,Model_category,Rev_per_unit


**Group by and visualizations**

In [ ]:
display(df.groupBy('Year','Branch_Name').agg(sum('Units_sold').alias('Total_units_sold')).sort('Year','Total_units_sold',ascending=[1,0]))

Year,Branch_Name,Total_units_sold


Databricks visualization. Run in Databricks to view.

**writing in to the silver container**

In [ ]:
df.write.format('parquet')\
             .option('header','true')\
             .mode('overwrite')\
            .save('abfss://silver@mynewadlsaccount.dfs.core.windows.net/carsales')

In [ ]:
%sql
select * from parquet.`abfss://silver@mynewadlsaccount.dfs.core.windows.net/carsales`

Branch_ID,Dealer_ID,Model_ID,Revenue,Units_sold,Date_Id,Day,Month,Year,Branch_Name,Dealer_Name,Model_category,Rev_per_unit
